# Chapter 4 - Interest rates

## Imports

In [18]:
import math

## Problems

### 4.1

In [19]:
quarterly_rate = 0.07
quarterly_compounded = math.pow(1 + quarterly_rate / 4, 4)

#### Continuous compounding

In [20]:
continuous_rate = math.log(quarterly_compounded)
print("continuous rate: ", continuous_rate)

continuous rate:  0.06939455333845228


#### Annually compounded

In [21]:
annual_rate = quarterly_compounded - 1
print("annual rate: ", annual_rate)

annual rate:  0.07185903128906279


### 4.2

In [22]:
semi_annual_rate = 0.05
continuous_rate = math.log((1 + semi_annual_rate / 2) ** 2)
print("continuous zero rate: ", continuous_rate)

continuous zero rate:  0.049385225180742925


#### Bond price

In [23]:
bond_yield = 0.052
half_year_yield = math.exp(-bond_yield / 2)
bond_price = 2 * half_year_yield * (1 + half_year_yield) + 102 * (half_year_yield ** 3)
print("bond price: ", bond_price)

bond price:  98.1936994203443


#### 18 month (continuous) zero rate

In [24]:
half_year_cost = math.exp(continuous_rate / 2)
half_year = 2 * half_year_cost
full_year = 2 * half_year_cost ** 2
extended_zero_rate = -math.log((bond_price - half_year - full_year) / 102) / 1.5
print("18 month zero rate: ", extended_zero_rate)

18 month zero rate:  0.0541510289146814


### 4.3

In [25]:
yearly_rate = 0.1
print("yearly rate: ", yearly_rate)
semi_annual_rate = (math.sqrt(1 + yearly_rate) - 1) * 2
print("semi-annual rate: ", semi_annual_rate)
monthly_rate = (math.pow(1 + yearly_rate, 1 / 12) - 1) * 12
print("monthly rate: ", monthly_rate)
continuous_rate = math.log(1 + yearly_rate)
print("continuous rate: ", continuous_rate)

yearly rate:  0.1
semi-annual rate:  0.09761769634030326
monthly rate:  0.09568968514684517
continuous rate:  0.09531017980432493


### 4.4

#### Risk-free rates

In [26]:
three_month_rate = 0.03
print("three month rate: ", three_month_rate)
six_month_rate = 0.032
print("six month rate: ", six_month_rate)
nine_month_rate = 0.034
print("nine month rate: ", nine_month_rate)
twelve_month_rate = 0.035
print("twelve month rate: ", twelve_month_rate)
fifteen_month_rate = 0.036
print("fifteen month rate: ", fifteen_month_rate)
eighteen_month_rate = 0.037
print("eighteen month rate: ", eighteen_month_rate)

three month rate:  0.03
six month rate:  0.032
nine month rate:  0.034
twelve month rate:  0.035
fifteen month rate:  0.036
eighteen month rate:  0.037


#### Forward rates

##### The forward rate calculation

In [27]:
def forward_rate(previous_rate: float, previous_duration: float, next_rate: float, duration_difference: float) -> float:
    '''
    All durations are in years.  duration_difference is the next duration minus the previous duration.  E.g., 
    for the forward rates between quarters, this would be 3 months or 1/4 of a year.
    '''
    return next_rate + (previous_duration / duration_difference) * (next_rate - previous_rate)

print("The results of Table 4.5 are replicated below using forward_rate.")
print("year 1 -> year 2 forward rate: ", forward_rate(0.03, 1, 0.04, 1))
print("year 2 -> year 3 forward rate: ", forward_rate(0.04, 2, 0.046, 1))
print("year 3 -> year 4 forward rate: ", forward_rate(0.046, 3, 0.05, 1))
print("year 4 -> year 5 forward rate: ", forward_rate(0.05, 4, 0.053, 1))  

The results of Table 4.5 are replicated below using forward_rate.
year 1 -> year 2 forward rate:  0.05
year 2 -> year 3 forward rate:  0.057999999999999996
year 3 -> year 4 forward rate:  0.06200000000000001
year 4 -> year 5 forward rate:  0.06499999999999997


##### The quarterly forward rates

In [30]:
quarter_duration = 1 / 4
print("quarter conversion to years: ", quarter_duration)
print("====================================")

print("quarter 2 forward rate: ", forward_rate(three_month_rate, quarter_duration, six_month_rate, quarter_duration))
print("quarter 3 forward rate: ", forward_rate(six_month_rate, 2 * quarter_duration, nine_month_rate, quarter_duration))
print("quarter 4 forward rate: ", forward_rate(nine_month_rate, 3 * quarter_duration, twelve_month_rate, quarter_duration))
print("quarter 5 forward rate: ", forward_rate(twelve_month_rate, 4 * quarter_duration, fifteen_month_rate, quarter_duration))
print("quarter 6 forward rate: ", forward_rate(fifteen_month_rate, 5 * quarter_duration, eighteen_month_rate, quarter_duration))

quarter conversion to years:  0.25
quarter 2 forward rate:  0.034
quarter 3 forward rate:  0.038000000000000006
quarter 4 forward rate:  0.038000000000000006
quarter 5 forward rate:  0.03999999999999997
quarter 6 forward rate:  0.042
